In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import input_file_name, row_number,when,col,min,max,unix_timestamp,rank, dense_rank,date_format,count,lead,lag,sum,expr,from_utc_timestamp,substring,abs,from_unixtime,to_date,greatest,to_timestamp,lit,concat,split,current_date, concat_ws,regexp_extract,lpad,first, last,date_add,hour,coalesce,floor,window
from functools import reduce
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("MulRead").getOrCreate()

25/03/11 21:06:59 WARN Utils: Your hostname, sushants-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.93 instead (on interface en0)
25/03/11 21:06:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/11 21:06:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/11 21:06:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
df2_block = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/Users/sushantshinde/workspaces/trvi/blockformer8-9/*.csv").withColumn("join_date", date_format(col("_time"), "yyyy-MM-dd HH:mm:ss").cast("timestamp"))\
    .withColumnRenamed("B1D1.AI_D1B31","AI_D1B31").withColumnRenamed("B1D1.AI_D1B32","AI_D1B32").withColumnRenamed("B1D1.AI_D1B34","AI_D1B34")\
    # .filter(col("join_date").cast("timestamp").between('2025-03-08T22:00:00','2025-03-09T22:00:00'))

In [25]:
# Convert _time to timestamp
df2_block = df2_block.withColumn("_time", to_timestamp("_time"))

# Create a window to track changes
window = Window.orderBy("_time")

# Use lag to compare current and previous values
df2_block = df2_block.withColumn("prev_valueblock1", lag("IO_MI_B1D1C3cut").over(window))
df2_block = df2_block.withColumn("prev_prev_valueblock1", lag("IO_MI_B1D1C3cut", 3).over(window))


# Create a new column 'start_time' in the original DataFrame
df2_block = df2_block.withColumn(
    "end_time_block1",
    # when(
    #     (col("IO_MI_B1D1C3cut") == 1) & (col("prev_value").isNull() | (col("prev_value") == 0)),
    #     col("_time")
    # ).otherwise(None)
     when(
        (col("IO_MI_B1D1C3cut") == 0) & (col("prev_valueblock1") == 1) 
        & (col('prev_prev_valueblock1') == 1)
        ,
        col("_time")
    ).otherwise(None)
)

df2_block = df2_block.withColumn("prev_valueblock2", lag("IO_MI_B1D2C3cut").over(window))
df2_block = df2_block.withColumn("prev_prev_valueblock2", lag("IO_MI_B1D2C3cut", 3).over(window))


# Create a new column 'start_time' in the original DataFrame
df2_block = df2_block.withColumn(
    "end_time_block2",
    # when(
    #     (col("IO_MI_B1D1C3cut") == 1) & (col("prev_value").isNull() | (col("prev_value") == 0)),
    #     col("_time")
    # ).otherwise(None)
     when(
        (col("IO_MI_B1D2C3cut") == 0) & (col("prev_valueblock2") == 1 ) 
        & (col('prev_prev_valueblock2') == 1)
        ,
        col("_time")
    ).otherwise(None)
)

# df2_block = df2_block.drop('prev_valueblock2','prev_valueblock1','prev_prev_valueblock1','prev_prev_valueblock2')

# Fill null values in 'start_time' column
# window_fill = Window.orderBy("_time").rowsBetween(Window.currentRow, Window.unboundedFollowing)
# df_with_start_time = df_with_start_time.withColumn(
#     "start_time",
#     last("start_time", True).over(window_fill)
# )

# Display the DataFrame with start times
# df2_block.show(100)


In [26]:
final_join3 = df2_block.withColumn('BlockEndTIme',
                                   when(col('end_time_block1').isNotNull(),col('end_time_block1'))
                                   .when(col('end_time_block2').isNotNull(),col('end_time_block2')).otherwise(None)
                                   )


df = final_join3.withColumn("BlockEndTIme", col("BlockEndTIme").cast("timestamp"))
window_spec = Window.orderBy("BlockEndTIme")

df = df.withColumn("next_datetime", lead("BlockEndTIme").over(window_spec))

df = df.withColumn("time_diff_hours", 
                   when(
                       col("next_datetime").isNotNull(), 
                       (unix_timestamp(col("next_datetime")) - unix_timestamp(col("BlockEndTIme"))) / 3600
                   ).otherwise(0)  
)

df = df.withColumn("is_new_block", when(col("time_diff_hours") > 3, 1).otherwise(0))
window_spec_cumsum = Window.orderBy("BlockEndTIme").rowsBetween(Window.unboundedPreceding, Window.currentRow)

df = df.withColumn("block_id", sum("is_new_block").over(window_spec_cumsum) + 1)
# df = df.withColumn("batch_id", sum("is_new_batch").over(window_spec_cumsum) + 1)


df =  df.withColumn('BlockFomerId',
                                   when(col('end_time_block1').isNotNull(),'BlockFormer1')
                                   .when(col('end_time_block2').isNotNull(),'BlockFormer2').otherwise(None)
                                   )
df = df.withColumn("block_id", 
                   when(
                       col("is_new_block") == 1, 
                        col("block_id") - 1  # Assign the previous block_id to the first row of the new block
                   ).otherwise(col("block_id"))
)
# .filter(col('block_id') == 2)
df = df.drop("next_datetime", "time_diff_hours", "is_new_block")

# df = df.filter(col('join_date').between('2025-03-08 22:12:28','2025-03-09 21:29:30'))


In [27]:
df.groupBy('block_id').agg(min('join_date'),max('join_date'),count('BlockFomerId')).show(truncate=False)
# df.show(100)

25/03/11 21:52:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/11 21:52:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/11 21:52:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/11 21:52:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/11 21:52:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/11 21:52:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/11 2

+--------+-------------------+-------------------+-------------------+
|block_id|min(join_date)     |max(join_date)     |count(BlockFomerId)|
+--------+-------------------+-------------------+-------------------+
|1       |2025-03-07 23:04:29|2025-03-11 09:31:48|2151               |
|2       |2025-03-09 01:23:21|2025-03-09 21:29:30|2106               |
|3       |2025-03-10 01:31:36|2025-03-10 01:52:35|8                  |
|4       |2025-03-10 05:28:01|2025-03-10 14:17:57|823                |
|5       |2025-03-11 08:21:46|2025-03-11 09:31:25|142                |
+--------+-------------------+-------------------+-------------------+



In [28]:
blob_path = '/Users/sushantshinde/workspaces/trvi/output'
df.repartition(1).orderBy('_time') \
    .write \
    .mode("overwrite") \
    .option("header", True) \
    .csv(blob_path)

25/03/11 21:52:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/11 21:52:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/11 21:52:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/11 21:52:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/11 21:52:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/11 21:52:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/11 2

In [10]:
# Read all CSV files in the folder
alfomatic_path = '/Users/sushantshinde/workspaces/trvi/alfomatic8-9/*.csv'
windowSpec_csv_filename = Window.partitionBy('FileName').orderBy("FileName")
df_alf = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(alfomatic_path).withColumn("FileName", input_file_name()).withColumn("join_date", date_format(col("_time"), "yyyy-MM-dd HH:mm:ss").cast("timestamp"))\
    .withColumn("FileName", input_file_name())\
    .withColumn("file_id", dense_rank().over(Window.orderBy("FileName")))

In [37]:
df_ids = df_alf.withColumn('VatNo1',when((col('V_G20EMP_N') == 1) | (col('V_G20EMP_N_1') == 1), col('_time')))\
                .withColumn('VatNo2',when((col('V_G20EMP_N') == 2) | (col('V_G20EMP_N_1') == 2), col('_time')))\
                .withColumn('VatNo3',when((col('V_G20EMP_N') == 3) | (col('V_G20EMP_N_1') == 3), col('_time')))\
                .withColumn('VatNo4',when((col('V_G20EMP_N') == 4) | (col('V_G20EMP_N_1') == 4), col('_time')))\
                .withColumn('VatNo5',when((col('V_G20EMP_N') == 5) | (col('V_G20EMP_N_1') == 5), col('_time')))\
                .withColumn('VatNo6',when((col('V_G20EMP_N') == 6) | (col('V_G20EMP_N_1') == 6), col('_time')))\
                .withColumn('VatNo7',when((col('V_G20EMP_N') == 7) | (col('V_G20EMP_N_1') == 7), col('_time')))\
                

In [39]:
df_ids = df_ids.withColumn('VATEndTIme',
                                   when(col('VatNo1').isNotNull(),col('VatNo1'))
                                   .when(col('VatNo2').isNotNull(),col('VatNo2'))
                                   .when(col('VatNo3').isNotNull(),col('VatNo3'))
                                   .when(col('VatNo4').isNotNull(),col('VatNo4'))
                                   .when(col('VatNo5').isNotNull(),col('VatNo5'))
                                   .when(col('VatNo6').isNotNull(),col('VatNo6'))
                                   .when(col('VatNo7').isNotNull(),col('VatNo7'))
                                   .otherwise(None)
                                   )

In [40]:
# df_group = final_join3.groupBy('BlockFomerId','file_id','V_G20EMP_N').agg(count("*").alias('blocks count')).filter((col('BlockFomerId').isNotNull()) & (col('V_G20EMP_N') != 0))\
blob_path = '/Users/sushantshinde/workspaces/trvi/output'
df_ids.repartition(1).orderBy('_time') \
    .write \
    .mode("overwrite") \
    .option("header", True) \
    .csv(blob_path)

25/03/11 11:35:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/11 11:35:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/11 11:35:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/11 11:35:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
